In [1]:
import dwdatareader as dw
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
import os
import re
import glob
import pandas as pd

In [2]:
def get_seam_endpoints(z_pos, fs):
    """Function that gets the endpoints of the different weld seams. The end is defined as the moment as the tool is moved
    up and released out of the metal sheet. For this purpose the signal is filtered by a FIR Filter too smooth the
    stair shaped signal of the roboter position in z. After this the second derivate of the position is calculated
    to get the first point of steppping out of the sheet.

    The signals are plotted and the result can be validated by visual control.

    The functions returns the times of the end of welds."""


    # creating and applying the fir filter
    taps = 100
    fir_filt = signal.firwin(taps, 1, fs=fs)
    filtered = signal.lfilter(fir_filt, 1.0, z_pos)

    # fir filter phase shifts the signal. delay determines the shift in seconds
    delay = 0.5 * (taps - 1) / fs

    # defining start point to look for the peak which correspondends to the out stepping of the tool
    start_offset = int(5 * fs)

    # getting the peaks
    peaks, properties = signal.find_peaks(np.gradient(np.gradient(filtered[start_offset:])), height=[0.002, 0.006],
                                          prominence=.004)

    peaks += start_offset

    # plot signals and results
    # fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

    # ax1.plot(time_z_pos, z_pos, marker="x", markersize=2, linestyle="None", linewidth=.5, color="black",
    #          label="Original Signal")
    # ax1.plot(time_z_pos - delay, filtered, linewidth=.5, color="blue", label="Filtered Signal")
    # ax1.legend(loc="upper right")
    # ax1.set_ylabel("Z-Position Roboter [mm]")

    # ax2.set_xlabel("Time (s)")
    # ax2.plot(time_z_pos - delay, np.gradient(np.gradient(filtered)), linewidth=.5, color="black",
    #          label="Second Derivate of filtered Signal")
    # ax2.legend(loc="upper right")

    # for i, peak in enumerate(peaks):
    #     ax1.axvline(time_z_pos[peak] - delay, color="red", linewidth=.5)
    #     ax2.axvline(time_z_pos[peak] - delay, color="red", linewidth=.5)

    # plt.show()

    # check if exactly two end points were found. If not exactly twi points were found an exception is raised
    if len(peaks) > 2:
        print("more than two maximums found. Getting endpoints aborted")
        raise
    elif len(peaks) < 2:
        print("not enough maximums found")
        raise

    return time_z_pos[peaks[0]], time_z_pos[peaks[1]]

In [3]:
# root = r"G:\AkoS_Messdaten\20220427"

# filenames = os.listdir(root)

# r = re.compile(r"WKZ_150600_(207\d{3})_(\d+)$")

In [4]:
# for filename in filenames:
#     if filename.split(".")[1] != "dxd":
#         continue

#     # check if a correct data name is used
#     if re.match(r, filename.split(".")[0]) is not None:
#         result = re.search(r, filename.split(".")[0])
#         wkz_name = result.group(1)
#         sample_no = result.group(2)
#     else:
#         continue

#filepath = os.path.join(root, filename)
filepath=r'..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin\data\Combined_folder\WKZ_150600_207003_1.dxd'
dewefile_handle = dw.open(filepath)

z_pos = dewefile_handle["Rob_Pos_Z"].series().to_numpy(dtype='float')
time_z_pos = dewefile_handle["Rob_Pos_Z"].series().index
time_z_pos -= time_z_pos[0]

fs_z_pos = dewefile_handle["Rob_Pos_Z"].number_of_samples / time_z_pos[-1]
print(fs_z_pos)
# print(filename.split(".")[0])

endpoint_L, endpoint_Maeander = get_seam_endpoints(z_pos, fs_z_pos)

print("{:.2f}, {:.2f}".format(endpoint_L, endpoint_Maeander))


125.01511019723515
27.17, 82.48


In [17]:
import os
directory=r'..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data'
filenames=[]
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.dxd'):
            filepath=os.path.join(root,file)
            filenames.append(filepath)

all_end_points=[]
for filepath in filenames:
    print(filepath)
    end_point=[]
    substring=filepath[89:]
    # n.append(substring)
    # name_list.append(n)
    dewefile_handle = dw.open(filepath)

    z_pos = dewefile_handle["Rob_Pos_Z"].series().to_numpy(dtype='float')
    time_z_pos = dewefile_handle["Rob_Pos_Z"].series().index
    time_z_pos -= time_z_pos[0]

    fs_z_pos = dewefile_handle["Rob_Pos_Z"].number_of_samples / time_z_pos[-1]
    #print(fs_z_pos)
    # print(filename.split(".")[0])

    endpoint_L, endpoint_Maeander = get_seam_endpoints(z_pos, fs_z_pos)
    end_point.append(endpoint_L)
    end_point.append(endpoint_Maeander)
    end_point.insert(0,substring)
    
    all_end_points.append(end_point)

..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data\WKZ_207003\WKZ_150600_207003_1.dxd
..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data\WKZ_207003\WKZ_150600_207005_24.dxd
..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data\WKZ_207003\WKZ_150600_207005_25.dxd
..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data\WKZ_207003\WKZ_150600_207011_19.dxd
..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data\WKZ_207003\WKZ_150600_207011_7.dxd
..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data\WKZ_207005\WKZ_150600_207005_0m Metallographie.dxd
..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data\WKZ_207005\WKZ_150600_207005_1.dxd
..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data\WKZ_207005\WKZ_150600_207005_10.dxd
..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\dat

In [16]:
for filepath in filenames:
    print(filepath[89:])

WKZ_150600_207003_1.dxd
WKZ_150600_207005_24.dxd
WKZ_150600_207005_25.dxd
WKZ_150600_207011_19.dxd
WKZ_150600_207011_7.dxd
WKZ_150600_207005_0m Metallographie.dxd
WKZ_150600_207005_1.dxd
WKZ_150600_207005_10.dxd
WKZ_150600_207005_11.dxd
WKZ_150600_207005_12.dxd
WKZ_150600_207005_13.dxd
WKZ_150600_207005_14.dxd
WKZ_150600_207005_15.dxd
WKZ_150600_207005_16.dxd
WKZ_150600_207005_17.dxd
WKZ_150600_207005_18.dxd
WKZ_150600_207005_19.dxd
WKZ_150600_207005_2.dxd
WKZ_150600_207005_20.dxd
WKZ_150600_207005_21.dxd
WKZ_150600_207005_22.dxd
WKZ_150600_207005_23.dxd
WKZ_150600_207005_24.dxd
WKZ_150600_207005_25.dxd
WKZ_150600_207005_3.dxd
WKZ_150600_207005_4.dxd
WKZ_150600_207005_5.dxd
WKZ_150600_207005_6.dxd
WKZ_150600_207005_7.dxd
WKZ_150600_207005_8.dxd
WKZ_150600_207005_9.dxd
WKZ_150600_207006_0m_Metallographie.dxd
WKZ_150600_207006_1.dxd
WKZ_150600_207006_10.dxd
WKZ_150600_207006_11.dxd
WKZ_150600_207006_12.dxd
WKZ_150600_207006_13.dxd
WKZ_150600_207006_14.dxd
WKZ_150600_207006_15.dxd
WKZ_150

In [18]:
all_end_points

[['WKZ_150600_207003_1.dxd', 27.166787941215095, 82.47630114990659],
 ['WKZ_150600_207005_24.dxd', 25.798842206412928, 81.05237234567574],
 ['WKZ_150600_207005_25.dxd', 25.838849945986794, 81.1643799102269],
 ['WKZ_150600_207011_19.dxd', 26.095087476950482, 81.32514791582746],
 ['WKZ_150600_207011_7.dxd', 25.839085694478708, 81.34116711103287],
 ['WKZ_150600_207005_0m Metallographie.dxd',
  26.446889332673976,
  82.06040027609792],
 ['WKZ_150600_207005_1.dxd', 26.014880958592016, 81.70041899923194],
 ['WKZ_150600_207005_10.dxd', 25.81487840989007, 81.50044090644224],
 ['WKZ_150600_207005_11.dxd', 25.9508080574044, 81.59471045840746],
 ['WKZ_150600_207005_12.dxd', 25.846854350938997, 81.19635404263295],
 ['WKZ_150600_207005_13.dxd', 25.991221322883348, 81.42039461196862],
 ['WKZ_150600_207005_14.dxd', 25.95885886374981, 81.43640710077159],
 ['WKZ_150600_207005_15.dxd', 25.966823941510654, 81.57437207931747],
 ['WKZ_150600_207005_16.dxd', 26.00210666029993, 81.3164234314547],
 ['WKZ_1506

In [19]:
end_seam_df = pd.DataFrame(all_end_points)

In [20]:
header_name=['Filename','L_end_point', 'Meander_end_point']
end_seam_df.to_csv('CSV\end_seam_points.csv',index=False,header=header_name)

In [ ]:
directory="..\Exchange_Preeti_Sarin\Exchange_Preeti_Sarin (1)\Exchange_Preeti_Sarin\data"
i=0
filenames=[]
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.dxd'):
            filepath=os.path.join(root,file)
            filenames.append(filepath)
 
print(len(filenames))

In [ ]:
all_end_points=[]
for filepath in filenames:
    end_point=[]
    substring=filepath[68:]
    # n.append(substring)
    # name_list.append(n)
    dewefile_handle = dw.open(filepath)

    z_pos = dewefile_handle["Rob_Pos_Z"].series().to_numpy(dtype='float')
    time_z_pos = dewefile_handle["Rob_Pos_Z"].series().index
    time_z_pos -= time_z_pos[0]

    fs_z_pos = dewefile_handle["Rob_Pos_Z"].number_of_samples / time_z_pos[-1]
    #print(fs_z_pos)
    # print(filename.split(".")[0])

    endpoint_L, endpoint_Maeander = get_seam_endpoints(z_pos, fs_z_pos)
    end_point.append(endpoint_L)
    end_point.append(endpoint_Maeander)
    end_point.insert(0,substring)
    
    all_end_points.append(end_point)

In [ ]:
end_seam_df = pd.DataFrame(all_end_points)

In [ ]:
header_name=['Filename','L_end_point', 'Meander_end_point']
end_seam_df.to_csv('CSV\end_seam_points.csv',index=False,header=header_name)